# Overlapとmp_term_nameの情報を統合する

In [1]:
# Move up to top directory
import os
os.getcwd()
os.chdir('../')
os.getcwd()

'/mnt/e/impc-genes_with_similar_phenotypes'

In [20]:
from pathlib import Path
from collections import defaultdict
from itertools import combinations
import csv
import pandas as pd
from matplotlib import pyplot as plt

In [21]:
path_overlap = Path("data", "overlap_ratios.csv")
path_mp = Path("data", "mp_term_name", "male_infertility.csv")

In [22]:
df_overlap = pd.read_csv(path_overlap)
df_mp = pd.read_csv(path_mp)

In [26]:
df_mp = df_mp[["marker_symbol", "effect_size"]].drop_duplicates()
df_mp

,marker_symbol,effect_size
0,Tex38,1.0
1,Pou2af3,1.0
2,Stoml2,1.0
3,Sbf1,1.0
4,Cyrib,1.0
...,...,...
320,Cep19,1.0
321,Bzw1,1.0
322,Septin10,1.0
323,Cfap57,1.0


In [85]:
print(df_overlap[df_overlap["intersect_count"] > 5 & df_overlap["overlap_ratio"] > 0.5])

TypeError: Cannot perform 'rand_' with a dtyped [float64] array and scalar of type [bool]

In [77]:
df_overlap_filterd = df_overlap[df_overlap["intersect_count"] > 5].reset_index(drop=True)

In [78]:
df_filtered = df_overlap_filterd[
    (df_overlap_filterd['marker1'].isin(df_mp['marker_symbol'])) &
    (df_overlap_filterd['marker2'].isin(df_mp['marker_symbol']))
]

In [79]:
df_merged = pd.merge(df_filtered, df_mp, how='inner', left_on='marker1', right_on='marker_symbol')

df_network = df_merged[["marker1", "marker2", "effect_size", "overlap_ratio"]]


In [80]:
df_network.columns = ["from", "to", "node_size", "edge_size"]
print(df_network)


        from       to  node_size  edge_size
0    Camsap3   Hectd3        1.0      0.219
1    Camsap3  Pou2af3        1.0      0.194
2    Camsap3    Spidr        1.0      0.250
3    Camsap3     Lepr        1.0      0.175
4    Camsap3   Prdm14        1.0      0.250
..       ...      ...        ...        ...
347   Ube2j2  Spata22        1.0      0.250
348   Ube2j2    Xrcc5        1.0      0.118
349   Ube2j2   Ythdc2        1.0      0.273
350   Ube2j2   Ube2j1        1.0      0.250
351      Cpe    Xrcc5        1.0      0.113

[352 rows x 4 columns]


In [81]:
df_network.to_csv(Path("data", "network.csv"), index=False)